## GSPT Steel plants

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
GSPT_path = "steel.xlsx"
import pandas as pd

In [31]:
df_gspt1= gspt.get_operating_plants(start_year=2019,end_year=2019,melt=False)
df_gspt=df_gspt1.drop_duplicates(["Coordinates"])
df_gspt1[["lat", "lon"]] = df_gspt1["Coordinates"].str.split(",",expand=True).astype(float)
len(df_gspt1)

848

In [28]:
from src.GSPTDataset import GSPTDataset
gspt = GSPTDataset(data_path="data\\raw_data\\Global-Steel-Plant-Tracker-2023-03-2.xlsx",
                        missing_years_path="data\\raw_data\\filled_missing_start_years.xlsx",
                        version_year=2023,
                        gspt2gspt_path="data\\raw_data\\GSPT2GSPT.json")
'''Database with operating steel plants in 2019'''
df_gspt= gspt.get_operating_plants(start_year=2019,end_year=2019,melt=True)
df_gspt=df_gspt.drop_duplicates(["Coordinates"])
df_gspt[["lat", "lon"]] = df_gspt["Coordinates"].str.split(",",expand=True).astype(float)

In [34]:
df_gspt[["lat", "lon"]] = df_gspt["Coordinates"].str.split(",",expand=True).astype(float)

In [50]:
import plotly.express as px

fig = px.scatter_mapbox(df_gspt, lat="lat", lon="lon", hover_name="Plant name (English)", hover_data=["Region", "Country"],
                        color_discrete_sequence=["fuchsia"], 
                        zoom=1, 
                        height=560,width=1000)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Steel Plants with available production

In [87]:
df=gspt.get_merged_capacity_prod(2019)
df=df.drop_duplicates(["Coordinates"])
df[["lat", "lon"]] = df["Coordinates"].str.split(",",expand=True).astype(float)
df=df[df['Crude steel production (ttpa)']>0]
df=df[["Plant name (English)_x","Region", "Country",'lat','lon','Crude steel production (ttpa)']]
df=df.rename(columns={'Crude steel production (ttpa)':'Production 2019'})

In [88]:
for year in [2020,2021]:
    df1=gspt.get_merged_capacity_prod(year)
    df1=df1.drop_duplicates(["Coordinates"])
    df1[["lat", "lon"]] = df1["Coordinates"].str.split(",",expand=True).astype(float)
    df1=df1[df1['Crude steel production (ttpa)']>0]
    df1=df1[["Plant name (English)_x","Region", "Country",'lat','lon','Crude steel production (ttpa)']]
    df1=df1.rename(columns={'Crude steel production (ttpa)':f'Production {year}'})
    
    df = pd.merge(df, df1, on=['Plant name (English)_x','Country','Region','lat','lon'], how='outer')

In [93]:
df

,Plant name (English)_x,Region,Country,lat,lon,Production 2019,Production 2020,Production 2021
0,ArcelorMittal Sonasid Casablanca steel plant,Africa,Morocco,33.136529,-8.603555,500.0,500.0,600.000000
1,ArcelorMittal Vanderbijlpark Steel Works,Africa,South Africa,-26.657367,27.822310,1900.0,1400.0,2000.000000
2,ArcelorMittal Newcastle Steel Works,Africa,South Africa,-27.705376,30.014063,1500.0,700.0,1000.000000
3,ArcelorMittal Saldanha Steel Works,Africa,South Africa,-32.979520,18.023228,900.0,NaN,NaN
4,BSRM Steels Chattogram plant,Asia Pacific,Bangladesh,22.381060,91.768620,175.0,103.0,515.000000
...,...,...,...,...,...,...,...,...
292,Cleveland-Cliffs Dearborn steel plant,North America,United States,42.308171,-83.161353,NaN,NaN,2018.389773
293,Steel Dynamics Columbus plant,North America,United States,33.447580,-88.572683,NaN,NaN,3094.000000
294,Cleveland-Cliffs Mansfield Works,North America,United States,40.790123,-82.526239,NaN,NaN,403.677955
295,U.S. Steel Edgar Thomson Plant,North America,United States,40.395140,-79.859645,NaN,NaN,1682.000000


In [89]:
two_year_prod= df[df['Production 2019'].notna() & df['Production 2020'].notna()]

In [96]:
two_year_prod.Country.unique()

array(['Morocco', 'South Africa', 'Bangladesh', 'China', 'India',
       'Indonesia', 'Japan', 'Malaysia', 'Pakistan', 'South Korea',
       'Thailand', 'Argentina', 'Brazil', 'Chile', 'Guatemala', 'Peru',
       'Kazakhstan', 'Russia', 'Belarus', 'Belgium',
       'Bosnia and Herzegovina', 'Bulgaria', 'Czech Republic', 'France',
       'Germany', 'Italy', 'Luxembourg', 'Netherlands', 'North Macedonia',
       'Romania', 'Spain', 'Ukraine', 'Qatar', 'Canada', 'Mexico',
       'United States'], dtype=object)

In [105]:
japon_production=two_year_prod.loc[(two_year_prod["Country"]=="Japan" )|( two_year_prod["Country"]=="South Korea")]
japon_production['variance'] = japon_production.apply(lambda row: 1 if row['Production 2020'] > row['Production 2019'] else 0, axis=1)
japon_production

,Plant name (English)_x,Region,Country,lat,lon,Production 2019,Production 2020,Production 2021,variance
84,Aichi Steel Chita Plant (Tokai),Asia Pacific,Japan,35.044558,136.900772,659.000000,847.000000,1045.000000,1
85,Daido Steel Chita plant (Tokai),Asia Pacific,Japan,35.016404,136.864744,978.000000,1255.000000,1410.000000,1
86,Nippon East Japan Works (Nagoya) steel plant,Asia Pacific,Japan,35.027575,136.870552,5848.000000,5569.000000,5657.000000,0
87,JFE West Japan Works (Fukuyama) steel plant,Asia Pacific,Japan,34.465222,133.431205,10563.913043,8817.391304,10173.913043,0
88,JFE West Japan Works (Kurashiki) steel plant,Asia Pacific,Japan,34.501198,133.722123,8126.086957,6782.608696,7826.086957,0
89,North Nippon Muroran Works steel plant,Asia Pacific,Japan,42.341873,140.997803,1433.000000,791.000000,1448.000000,0
92,Nippon Steel Wakayama Area (Sakai) plant,Asia Pacific,Japan,34.598220,135.454390,1200.000000,1090.000000,NaN,0
93,Nakayama Steel Works Osaka,Asia Pacific,Japan,34.628162,135.459688,542.000000,497.000000,NaN,0
94,"Nippon Kansai Works (Wakayama Area, Wakayama) ...",Asia Pacific,Japan,34.233519,135.135443,4324.000000,2694.000000,2732.000000,0
95,JFE East Japan Works (Chiba) steel plant,Asia Pacific,Japan,35.580917,140.104196,3883.381924,3516.034985,3830.903790,0


In [102]:
japon_production.to_csv("japon_producton.csv",index=False)

In [92]:
two_year_prod.loc[two_year_prod["Country"]=="Mexico"]

,Plant name (English)_x,Region,Country,lat,lon,Production 2019,Production 2020,Production 2021
205,Altos Hornos De Mexico S.A. (AHMSA) steel plant,North America,Mexico,26.876889,-101.416612,3550.0,1890.0,2000.0
206,Gerdau Corsa Ciudad Sahagún steel plant,North America,Mexico,19.753326,-98.596483,490.0,590.0,670.0
207,ArcelorMittal Lázaro Cárdenas steel plant,North America,Mexico,17.930688,-102.201498,3700.0,3600.0,3700.0


In [80]:
import plotly.express as px

fig = px.scatter_mapbox(two_year_prod, lat="lat", lon="lon", hover_name="Plant name (English)_x", hover_data=["Region", "Country"],
                        color_discrete_sequence=["fuchsia"], 
                        zoom=1, 
                        height=560,width=1000)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Oxford Steel Plants

In [ ]:
STEEL_PATH = "C:/Users/hamada.saleh/Documents/Python Scripts/bottom_up_alignment/data/raw/SFI-Global-Steel-Database-July-2021.xlsx"

In [ ]:
steel = pd.read_excel(STEEL_PATH, sheet_name="SFI_ALD_Steel_Database")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/hamada.saleh/Documents/Python Scripts/bottom_up_alignment/data/raw/SFI-Global-Steel-Database-July-2021.xlsx'

In [ ]:
steel.head()

,uid,city,state,country,iso3,country_code,region,sub_region,latitude,longitude,...,parent_holding_status,parent_ticker,parent_exchange,parent_permid_2,parent_name_2,ownership_stake_2,parent_lei_2,parent_holding_status_2,parent_ticker_2,parent_exchange_2
0,GASTAGO0001,Dande,Bengo Province,Angola,AGO,24,Africa,Sub-Saharan Africa,-8.582400,13.405271,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GASTALB0001,Rrethi i Elbasanit,Qarku i Elbasanit,Albania,ALB,8,Europe,Southern Europe,41.094611,20.026117,...,Private,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GASTARE0001,Abu Dhabi,NaN,United Arab Emirates,ARE,784,Asia,Western Asia,24.320682,54.480176,...,Private,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GASTARE0002,Abu Dhabi,Abu Dhabi,United Arab Emirates,ARE,784,Asia,Western Asia,24.322916,54.470987,...,Private,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GASTARE0003,Abu Dhabi,Abu Dhabi,United Arab Emirates,ARE,784,Asia,Western Asia,24.378720,54.475153,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
steel.columns

Index(['uid', 'city', 'state', 'country', 'iso3', 'country_code', 'region',
       'sub_region', 'latitude', 'longitude', 'accuracy', 'status',
       'plant_type', 'primary_production_type', 'primary_product', 'capacity',
       'capacity_source', 'year', 'owner_permid', 'owner_name', 'owner_source',
       'parent_permid', 'parent_name', 'ownership_stake', 'parent_lei',
       'parent_holding_status', 'parent_ticker', 'parent_exchange',
       'parent_permid_2', 'parent_name_2', 'ownership_stake_2', 'parent_lei_2',
       'parent_holding_status_2', 'parent_ticker_2', 'parent_exchange_2'],
      dtype='object')

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(steel, lat="latitude", lon="longitude", hover_name="uid", hover_data=["city", "country"],
                        color_discrete_sequence=["fuchsia"], 
                        zoom=1, 
                        height=560,width=1000,
                        labels={"title" :"Oxford steel plants"})
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Cement plants

In [ ]:
CEMENT_PATH = "C:/Users/hamada.saleh/Documents/Python Scripts/bottom_up_alignment/data/raw/SFI-Global-Cement-Database-July-2021.xlsx"

In [ ]:
cement = pd.read_excel(CEMENT_PATH, sheet_name="SFI_ALD_Cement_Database")

In [ ]:
cement.head()

,uid,city,state,country,iso3,country_code,region,sub_region,latitude,longitude,...,parent_holding_status,parent_ticker,parent_exchange,parent_permid_2,parent_name_2,ownership_stake_2,parent_lei_2,parent_holding_status_2,parent_ticker_2,parent_exchange_2
0,GACTAFG0001,Pol-e Khomri,Baghlan,Afghanistan,AFG,4.0,Asia,Southern Asia,35.965800,68.686338,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GACTAFG0002,Injil,Herat,Afghanistan,AFG,4.0,Asia,Southern Asia,34.322144,61.953503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,GACTAGO0001,Luanda,Luanda,Angola,AGO,24.0,Africa,Sub-Saharan Africa,-8.766173,13.316051,...,Private,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GACTAGO0002,Cacuaco,Luanda,Angola,AGO,24.0,Africa,Sub-Saharan Africa,-8.796392,13.426780,...,Private,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,GACTAGO0003,Lobito,Benguela,Angola,AGO,24.0,Africa,Sub-Saharan Africa,-12.342644,13.581766,...,Private,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
cement.columns

Index(['uid', 'city', 'state', 'country', 'iso3', 'country_code', 'region',
       'sub_region', 'latitude', 'longitude', 'accuracy', 'status',
       'plant_type', 'production_type', 'capacity', 'capacity_source', 'year',
       'owner_permid', 'owner_name', 'owner_source', 'parent_permid',
       'parent_name', 'ownership_stake', 'parent_lei', 'parent_holding_status',
       'parent_ticker', 'parent_exchange', 'parent_permid_2', 'parent_name_2',
       'ownership_stake_2', 'parent_lei_2', 'parent_holding_status_2',
       'parent_ticker_2', 'parent_exchange_2'],
      dtype='object')

In [ ]:
import plotly.express as px

fig = px.scatter_mapbox(cement, lat="latitude", lon="longitude", hover_name="uid", hover_data=["city", "country"],
                        color_discrete_sequence=["fuchsia"], 
                        zoom=1, 
                        height=560,width=1000)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()